In [36]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score, recall_score
import numpy as np




In [37]:
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)

current_dir = project_root = Path.cwd()

project_root = current_dir.parent

data_path = project_root/'data'/'training_set.csv'

print(data_path)


df_training


/Users/michaelwhitfield/school-coding-projects/mgmt-474/ccac-competion/data/training_set.csv


,CustomerID,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion,E_InstitutionName,E_InstitutionDMACode,E_InstitutionLatitude,E_InstitutionLongitude,E_InstitutionConference,E_InstitutionEnrollment_Male,E_InstitutionEnrollment_Female,E_InstitutionEnrollment_Total,E_InstitutionNCAAMemberSinceDate,E_RegularSeasonWins,E_RegularSeasonLosses,E_RegularSeasonAverageAttendance,E_RegularSeasonAverageScore,E_Rk,E_Seed_Rank,E_NetRtg,E_Luck,M_InstitutionName,M_InstitutionDMACode,M_InstitutionLatitude,M_InstitutionLongitude,M_InstitutionConference,M_InstitutionEnrollment_Male,M_InstitutionEnrollment_Female,M_InstitutionEnrollment_Total,M_InstitutionNCAAMemberSinceDate,M_RegularSeasonWins,M_RegularSeasonLosses,M_RegularSeasonAverageAttendance,M_RegularSeasonAverageScore,M_Rk,M_Seed_Rank,M_NetRtg,M_Luck,S_InstitutionName,S_InstitutionDMACode,S_InstitutionLatitude,S_InstitutionLongitude,S_InstitutionConference,S_InstitutionEnrollment_Male,S_InstitutionEnrollment_Female,S_InstitutionEnrollment_Total,S_InstitutionNCAAMemberSinceDate,S_RegularSeasonWins,S_RegularSeasonLosses,S_RegularSeasonAverageAttendance,S_RegularSeasonAverageScore,S_Rk,S_Seed_Rank,S_NetRtg,S_Luck,W_InstitutionName,W_InstitutionDMACode,W_InstitutionLatitude,W_InstitutionLongitude,W_InstitutionConference,W_InstitutionEnrollment_Male,W_InstitutionEnrollment_Female,W_InstitutionEnrollment_Total,W_InstitutionNCAAMemberSinceDate,W_RegularSeasonWins,W_RegularSeasonLosses,W_RegularSeasonAverageAttendance,W_RegularSeasonAverageScore,W_Rk,W_Seed_Rank,W_NetRtg,W_Luck,m_win_%,s_win_%,e_win_%,w_win_%,E_distance,W_distance,M_distance,S_distance,E_dist_score
0,47028,32.5622,-86.0994,698.0,MONTGOMERY (SELMA),2021-12-25,1723503,2024-03-19 10:27:15 -0400,164,29,317,694,164,694,694,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,1,1,36.43,0.037,Tennessee,557,35.95208,-83.92585,Southeastern Conference,15269,18536,33805,09/01/1909,24,8,16065.47,79.47,5,2,26.61,-0.026,James Madison,569,38.43631,-78.87048,Sun Belt Conference,8342,12004,20346,09/01/1969,31,3,4471.35,84.35,63,12,12.42,0.093,Arizona,789,32.23267,-110.95080,Pac-12 Conference,13090,17292,30382,09/01/1936,25,8,12118.18,87.94,6,2,26.55,-0.047,0.750000,0.911765,0.911765,0.757576,1597.133829,2328.202760,426.563483,923.981672,0.000626
1,3511,42.8256,-86.0104,563.0,GRAND RAPIDS - KALMZOO - B. CRK,2021-04-02,963479,2024-03-18 10:16:39 -0400,164,51,334,328,164,334,164,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,1,1,36.43,0.037,Kansas,616,38.95855,-95.24757,Big 12 Conference,8859,9845,18704,09/01/1908,22,10,13952.66,75.25,27,4,17.94,0.058,Kentucky,541,38.03891,-84.50475,Southeastern Conference,9596,13127,22723,09/01/1936,23,9,17427.94,89.44,23,3,19.29,-0.040,Baylor,625,31.54687,-97.12104,Big 12 Conference,6008,8877,14885,09/01/1922,23,10,9517.79,80.55,15,3,21.90,-0.016,0.687500,0.718750,0.911765,0.696970,1135.742592,1591.208951,886.829991,547.271060,0.000880
2,58445,38.8808,-77.1129,511.0,"WASHINGTON, DC (HAGRSTWN)",2021-04-02,810038,2024-03-18 00:21:47 -0400,164,51,288,559,164,559,559,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,1,1,36.43,0.037,Purdue,582,40.42821,-86.91444,Big Ten Conference,21670,16279,37949,09/01/1914,29,4,13329.06,83.39,3,1,30.62,0.048,Houston,618,29.72039,-95.34354,Big 12 Conference,18290,19653,37943,09/01/1949,30,4,9347.35,73.03,2,1,31.17,0.042,Baylor,625,31.54687,-97.12104,Big 12 Conference,6008,8877,14885,09/01/1922,23,10,9517.79,80.55,15,3,21.90,-0.016,0.878788,0.882353,0.911765,0.696970,525.083169,1986.571818,856.081303,1954.472608,0.001904
3,28833,29.4969,-98.4032,641.0,SAN ANTONIO,2023-11-16,3384825,2024-03-21 1

In [38]:
#Baseline Model -- Choose Team with Higher Winning Percentage

#Create Dataset including information on Midwest Team and South Team

mid_south_basic_df = df_training.loc[:, ["RegionWinner_South", 'RegionWinner_Midwest', 'm_win_%','s_win_%', 'SemifinalWinner_South_Midwest']]

X_sm = mid_south_basic_df.drop(columns = ['SemifinalWinner_South_Midwest'])
Y_sm = mid_south_basic_df['SemifinalWinner_South_Midwest']

#Predict that the User will choose the team with higer win %
X_sm['Predicted Winner Binary'] =  (X_sm['m_win_%'] > X_sm['s_win_%']).astype(int)
X_sm['Predicted Winner'] =  np.where(X_sm['Predicted Winner Binary'] == 1, X_sm['RegionWinner_Midwest'], X_sm['RegionWinner_South'])
X_sm['Actual Winner'] = (mid_south_basic_df['SemifinalWinner_South_Midwest'] == X_sm['RegionWinner_Midwest']).astype(int)


recall = recall_score(X_sm['Actual Winner'], X_sm['Predicted Winner Binary'])
accuracy = accuracy_score(X_sm['Actual Winner'], X_sm['Predicted Winner Binary'])

print(f'Baseline Model: Recall {recall}, Accuracy {accuracy}')





Baseline Model: Recall 0.49974409205626447, Accuracy 0.5992600113844403


In [39]:
east_west_basic_df = df_training.loc[:, ["RegionWinner_East", 'RegionWinner_West', 'e_win_%','w_win_%', 'SemifinalWinner_East_West']]

X2_sm = east_west_basic_df.drop(columns = ['SemifinalWinner_East_West'])
Y2_sm = east_west_basic_df['SemifinalWinner_East_West']

#Predict that the User will choose the team with higher win %
X2_sm['Predicted Winner Binary'] =  (X2_sm['e_win_%'] > X2_sm['w_win_%']).astype(int) #IF east wins then 1, else 0
X2_sm['Predicted Winner'] =  np.where(X2_sm['Predicted Winner Binary'] == 1, X2_sm['RegionWinner_East'], X2_sm['RegionWinner_West'])
X2_sm['Actual Winner'] = (east_west_basic_df['SemifinalWinner_East_West'] == X2_sm['RegionWinner_East']).astype(int)


recall2 = recall_score(X2_sm['Actual Winner'], X2_sm['Predicted Winner Binary'])
accuracy2 = accuracy_score(X2_sm['Actual Winner'], X2_sm['Predicted Winner Binary'])

print(f'Baseline Model: Recall {recall2}, Accuracy {accuracy2}')




Baseline Model: Recall 0.8871265933638222, Accuracy 0.655259149859233


In [40]:
nat_champ_basic_df = nat_champ_basic_df = pd.DataFrame({
    "ew_pred_win_%": east_west_basic_df["e_win_%"].where(X2_sm["Predicted Winner Binary"] == 1, east_west_basic_df["w_win_%"]),
    "ms_pred_win_%": mid_south_basic_df["m_win_%"].where(X_sm["Predicted Winner Binary"] == 1, mid_south_basic_df["s_win_%"]),
    "actual_nat_champ": df_training['NationalChampion']  # Example actual champions
})

# Predict the national champion based on the higher winning percentage
nat_champ_basic_df["predict_nat_champ_binary"] = (
    nat_champ_basic_df["ew_pred_win_%"] > nat_champ_basic_df["ms_pred_win_%"]
).astype(int)

nat_champ_basic_df["actual_nat_champ_binary"] = (nat_champ_basic_df["actual_nat_champ"] == X2_sm["Predicted Winner"]).astype(int) #IF Pred Winner is East West

# Calculate accuracy and recall for national champion prediction
recall_nat_champ = recall_score(nat_champ_basic_df["actual_nat_champ_binary"], nat_champ_basic_df["predict_nat_champ_binary"])
accuracy_nat_champ = accuracy_score(nat_champ_basic_df["actual_nat_champ_binary"], nat_champ_basic_df["predict_nat_champ_binary"])

# Display results
print(f'National Champion Prediction - Baseline Model: Recall {recall_nat_champ:.2f}, Accuracy {accuracy_nat_champ:.2f}')

# X_train, X_test, y_train, y_test = train_test_split(X_sm, Y_sm, test_size=.2, random_state=24)

# log_reg_sm = LogisticRegressionCV(cv=5, scoring='recall', n_jobs=-1, refit=False, random_state=24)
# log_reg_sm.fit(X_train, y_train)


National Champion Prediction - Baseline Model: Recall 0.83, Accuracy 0.52
